In [2]:
%matplotlib notebook
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

import sys
import dtreeviz
import graphviz

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [3]:
df_mrt = pd.read_csv('auxiliary-data/sg-mrt-stations.csv')
df_com = pd.read_csv('auxiliary-data/sg-commerical-centres.csv')
df_ps = pd.read_csv('auxiliary-data/sg-primary-schools.csv')
df_ss = pd.read_csv('auxiliary-data/sg-secondary-schools.csv')
df_sm = pd.read_csv('auxiliary-data/sg-shopping-malls.csv')
df_sz = pd.read_csv('auxiliary-data/sg-subzones.csv')

In [4]:
df_sz.head()

,name,area_size,population,planning_area
0,ang mo kio town centre,0.3169,4810,ang mo kio
1,cheng san,0.9557,28070,ang mo kio
2,chong boon,1.0786,26500,ang mo kio
3,kebun bahru,1.0464,22620,ang mo kio
4,sembawang hills,0.8945,6850,ang mo kio


In [5]:
df_sz = df_sz.drop(columns=['area_size', 'planning_area'])
df_sm = df_sm.drop(columns=['lat', 'lng', 'planning_area'])
df_ss = df_ss.drop(columns=['lat', 'lng', 'planning_area'])
df_ps = df_ps.drop(columns=['lat', 'lng', 'planning_area'])
df_com = df_com.drop(columns=['lat', 'lng', 'planning_area', 'type'])
df_mrt = df_mrt.drop(columns=['lat', 'lng', 'planning_area', 'opening_year', 'name', 'line'])

In [6]:
df_sm.head()

,name,subzone
0,10 AM,marina south
1,313@Somerset,somerset
2,Aperia,kallang bahru
3,Balestier Hill Shopping Centre,balestier
4,Bugis Cube,bugis


In [7]:
mrt_count = df_mrt.groupby(["subzone"])["code"].count().reset_index(name="count")
com_count = df_com.groupby(["subzone"])["name"].count().reset_index(name="count")
ps_count = df_ps.groupby(["subzone"])["name"].count().reset_index(name="count")
ss_count = df_ss.groupby(["subzone"])["name"].count().reset_index(name="count")
sm_count = df_sm.groupby(["subzone"])["name"].count().reset_index(name="count")

In [8]:
com_count.head()

,subzone,count
0,alexandra hill,1
1,bishan east,1
2,bras basah,1
3,changi airport,3
4,cleantech,1


In [9]:
mrt_count.columns = ['subzone', 'mrt_count']
ps_count.columns = ['subzone', 'ps_count']
ss_count.columns = ['subzone', 'ss_count']
sm_count.columns = ['subzone', 'sm_count']
df_sz.columns = ['subzone', 'population']
com_count.columns = ['subzone', 'com_count']

In [10]:
df_sz.head()

,subzone,population
0,ang mo kio town centre,4810
1,cheng san,28070
2,chong boon,26500
3,kebun bahru,22620
4,sembawang hills,6850


In [11]:
mrt_count = mrt_count.sort_values(by='subzone', ascending=True)
ps_count = ps_count.sort_values(by='subzone', ascending=True)
ss_count = ss_count.sort_values(by='subzone', ascending=True)
sm_count = sm_count.sort_values(by='subzone', ascending=True)
df_sz = df_sz.sort_values(by='subzone', ascending=True)
com_count = com_count.sort_values(by='subzone', ascending=True)

In [12]:
frames = [mrt_count, ps_count, ss_count, sm_count, df_sz]

sz_mrt = df_sz.join(mrt_count.set_index('subzone'), on='subzone')
sz_mrt_ps = sz_mrt.join(ps_count.set_index('subzone'), on='subzone')
sz_ps_mrt_ss = sz_mrt_ps.join(ss_count.set_index('subzone'), on='subzone')
sz_ps_mrt_ss_sm = sz_ps_mrt_ss.join(sm_count.set_index('subzone'), on='subzone')
sz_ps_mrt_ss_sm_com = sz_ps_mrt_ss_sm.join(com_count.set_index('subzone'), on='subzone')

In [13]:
sz_ps_mrt_ss_sm_com.head()

,subzone,population,mrt_count,ps_count,ss_count,sm_count,com_count
236,admiralty,13920,NaN,1.0,NaN,NaN,NaN
185,airport road,0,NaN,NaN,NaN,NaN,NaN
36,alexandra hill,13490,NaN,NaN,1.0,1.0,1.0
37,alexandra north,2620,NaN,1.0,1.0,NaN,NaN
101,aljunied,39990,3.0,2.0,2.0,NaN,NaN


In [14]:
df_data = sz_ps_mrt_ss_sm_com  
df_data.info()  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 331 entries, 236 to 135
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   subzone     331 non-null    object 
 1   population  331 non-null    int64  
 2   mrt_count   102 non-null    float64
 3   ps_count    109 non-null    float64
 4   ss_count    101 non-null    float64
 5   sm_count    98 non-null     float64
 6   com_count   31 non-null     float64
dtypes: float64(5), int64(1), object(1)
memory usage: 20.7+ KB


In [15]:
df_data = df_data[df_data["population"] > 0] 
df_data = df_data.fillna(0) 
df_data.head()

,subzone,population,mrt_count,ps_count,ss_count,sm_count,com_count
236,admiralty,13920,0.0,1.0,0.0,0.0,0.0
36,alexandra hill,13490,0.0,0.0,1.0,1.0,1.0
37,alexandra north,2620,0.0,1.0,1.0,0.0,0.0
101,aljunied,39990,3.0,2.0,2.0,0.0,0.0
60,anak bukit,21870,1.0,2.0,0.0,3.0,0.0


In [16]:
from scipy.stats import norm, skew 

numeric_feats = df_data.dtypes[df_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = df_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)


Skew in numerical features: 



,Skew
sm_count,4.351365
com_count,3.857349
population,2.231436
ps_count,2.211657
ss_count,1.865074
mrt_count,1.750147


In [17]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features, and we will use box cox to transform them.".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    df_data[feat] = boxcox1p(df_data[feat], lam)

There are 6 skewed numerical features, and we will use box cox to transform them.


In [18]:
numeric_feats = df_data.dtypes[df_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = df_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)


Skew in numerical features: 



,Skew
com_count,3.139684
sm_count,1.743685
ps_count,1.182077
mrt_count,1.170277
ss_count,1.165716
population,-0.417021


In [19]:
y = df_data[['population']]
X = df_data.drop(columns=['population', 'subzone'])

In [20]:

iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(X)

mask = yhat != -1

mask = mask.reshape(-1, 1)
print(mask.shape)
print(y.shape)
X = X.iloc[mask,:]
y = y.iloc[mask]

(286, 1)
(286, 1)


In [21]:
y[y < 0] = 0
y.replace([np.inf, -np.inf], 0, inplace=True)
print(y)

     population
236   21.224026
37    15.044282
245   26.927118
0     17.115096
88     6.221214
..          ...
328   28.115443
329   26.276767
330   27.504543
125   23.742203
126   22.593141

[257 rows x 1 columns]


In [22]:
print(X.shape)
print(y.shape)

(257, 5)
(257, 1)


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [24]:
# Fitting the model
pop_model = LinearRegression()
pop_model.fit(X_train, y_train)

# Returning the R^2 for the model
pop_r2 = pop_model.score(X_train, y_train)
print('If R2 reduced, then the data become more linear')
print('R^2: {0}'.format(pop_r2)) # raw: R^2: 0.7643993036884857

If R2 reduced, then the data become more linear
R^2: 0.5377753363476623


In [25]:

coefficients = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(pop_model.coef_))], axis = 1)
print(coefficients)

           0          0
0  mrt_count  -0.430017
1   ps_count   7.608144
2   ss_count   3.901738
3   sm_count   2.308449
4  com_count -10.382854


In [26]:
def calculate_residuals(model, features, label):
    """
    Creates predictions on the features with the model and calculates residuals
    """
    predictions = model.predict(features)
    print(label.squeeze().shape)
    df_results = pd.DataFrame({'Truth': label.squeeze(), 'Predicted': predictions.squeeze()})
    df_results['Residuals'] = abs(df_results['Truth']) - abs(df_results['Predicted'])
    
    
    
    return df_results

In [27]:
def linear_assumption(model, features, label):
    """
    Linearity: Assumes that there is a linear relationship between the predictors and
               the response variable. If not, either a quadratic term or another
               algorithm should be used.
    """
    print('Assumption 1: Linear Relationship between the Target and the Feature', '\n')
        
    print('Checking with a scatter plot of actual vs. predicted.',
           'Predictions should follow the diagonal line.')
    
    # Calculating residuals for the plot
    df_results = calculate_residuals(model, features, label)
    
    # Plotting the actual vs predicted values
    sns.lmplot(x='Truth', y='Predicted', data=df_results, fit_reg=False, size=7)
        
    # Plotting the diagonal line
    line_coords = np.arange(df_results.min().min(), df_results.max().max())
    plt.plot(line_coords, line_coords,  # X and y points
             color='darkorange', linestyle='--')
    plt.title('Truth vs. Predicted')
    plt.show()

In [28]:
X_train = X.to_numpy()
y_train = y.to_numpy()
linear_assumption(pop_model, X_test, y_test)

Assumption 1: Linear Relationship between the Target and the Feature 

Checking with a scatter plot of actual vs. predicted. Predictions should follow the diagonal line.
(52,)


NameError: name 'sns' is not defined

In [ ]:
# https://stattrek.com/regression/linear-transformation#:~:text=In%20regression%2C%20a%20transformation%20to,linear%20relationship%20between%20two%20variables.



In [ ]:
def normal_errors_assumption(model, features, label, p_value_thresh=0.05):
    """
    Normality: Assumes that the error terms are normally distributed. If they are not,
    nonlinear transformations of variables may solve this.
               
    This assumption being violated primarily causes issues with the confidence intervals
    """
    from statsmodels.stats.diagnostic import normal_ad
    print('Assumption 2: The error terms are normally distributed', '\n')
    
    # Calculating residuals for the Anderson-Darling test
    df_results = calculate_residuals(model, features, label)
    
    print('Using the Anderson-Darling test for normal distribution')

    # Performing the test on the residuals
    p_value = normal_ad(df_results['Residuals'])[1]
    print('p-value from the test - below 0.05 generally means non-normal:', p_value)
    
    # Reporting the normality of the residuals
    if p_value < p_value_thresh:
        print('Residuals are not normally distributed')
    else:
        print('Residuals are normally distributed')
    
    # Plotting the residuals distribution
    plt.subplots(figsize=(12, 6))
    plt.title('Distribution of Residuals')
    sns.distplot(df_results['Residuals'])
    plt.show()
    
    print()
    if p_value > p_value_thresh:
        print('Assumption satisfied')
    else:
        print('Assumption not satisfied')
        print()
        print('Confidence intervals will likely be affected')
        print('Try performing nonlinear transformations on variables')

In [ ]:
normal_errors_assumption(pop_model, X_train, y_train)

In [ ]:
def multicollinearity_assumption(model, features, label, feature_names=None):
    """
    Multicollinearity: Assumes that predictors are not correlated with each other. If there is
                       correlation among the predictors, then either remove prepdictors with high
                       Variance Inflation Factor (VIF) values or perform dimensionality reduction
                           
                       This assumption being violated causes issues with interpretability of the 
                       coefficients and the standard errors of the coefficients.
    """
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    print('Assumption 3: Little to no multicollinearity among predictors')
        
    # Plotting the heatmap
    plt.figure(figsize = (10,8))
    sns.heatmap(pd.DataFrame(features, columns=feature_names).corr(), annot=True)
    plt.title('Correlation of Variables')
    plt.show()
        
    print('Variance Inflation Factors (VIF)')
    print('> 10: An indication that multicollinearity may be present, then we need to throw away that variable')
    print('-------------------------------------')
       
    # Gathering the VIF for each variable
    VIF = [variance_inflation_factor(features, i) for i in range(features.shape[1])]
    for idx, vif in enumerate(VIF):
        print('{0}: {1}'.format(feature_names[idx], vif))
        
    # Gathering and printing total cases of possible or definite multicollinearity
    possible_multicollinearity = sum([1 for vif in VIF if vif > 10])
    definite_multicollinearity = sum([1 for vif in VIF if vif > 100])
    print()
    print('{0} cases of possible multicollinearity'.format(possible_multicollinearity))
    print('{0} cases of definite multicollinearity'.format(definite_multicollinearity))
    print()

    if definite_multicollinearity == 0:
        if possible_multicollinearity == 0:
            print('Assumption satisfied')
        else:
            print('Assumption possibly satisfied')
            print()
            print('Coefficient interpretability may be problematic')
            print('Consider removing variables with a high Variance Inflation Factor (VIF)')

    else:
        print('Assumption not satisfied')
        print()
        print('Coefficient interpretability will be problematic')
        print('Consider removing variables with a high Variance Inflation Factor (VIF)')

In [ ]:
multicollinearity_assumption(pop_model, X_train, y_train, [ 'ps_counts',  'ss_count', 'mrt_counts', 'sm_count', 'com_count'])

In [ ]:
def autocorrelation_assumption(model, features, label):
    """
    Autocorrelation: Assumes that there is no autocorrelation in the residuals. If there is
                     autocorrelation, then there is a pattern that is not explained due to
                     the current value being dependent on the previous value.
                     This may be resolved by adding a lag variable of either the dependent
                     variable or some of the predictors.
    """
    from statsmodels.stats.stattools import durbin_watson
    print('Assumption 4: No Autocorrelation', '\n')
    
    # Calculating residuals for the Durbin Watson-tests
    df_results = calculate_residuals(model, features, label)

    print('\nPerforming Durbin-Watson Test')
    print('Values of 1.5 < d < 2.5 generally show that there is no autocorrelation in the data')
    print('0 to 2< is positive autocorrelation')
    print('>2 to 4 is negative autocorrelation')
    print('-------------------------------------')
    durbinWatson = durbin_watson(df_results['Residuals'])
    print('Durbin-Watson:', durbinWatson)
    if durbinWatson < 1.5:
        print('Signs of positive autocorrelation', '\n')
        print('Assumption not satisfied')
    elif durbinWatson > 2.5:
        print('Signs of negative autocorrelation', '\n')
        print('Assumption not satisfied')
    else:
        print('Little to no autocorrelation', '\n')
        print('Assumption satisfied')

In [ ]:
autocorrelation_assumption(pop_model, X_train, y_train)

In [ ]:
def homoscedasticity_assumption(model, features, label):
    """
    Homoscedasticity: Assumes that the errors exhibit constant variance
    """
    print('Assumption 5: Homoscedasticity of Error Terms', '\n')
    
    print('Residuals should have relative constant variance')
        
    # Calculating residuals for the plot
    df_results = calculate_residuals(model, features, label)

    # Plotting the residuals
    plt.subplots(figsize=(12, 6))
    ax = plt.subplot(111)  # To remove spines
    plt.scatter(x=df_results.index, y=df_results.Residuals, alpha=0.5)
    plt.plot(np.repeat(0, df_results.index.max()), color='darkorange', linestyle='--')
    ax.spines['right'].set_visible(False)  # Removing the right spine
    ax.spines['top'].set_visible(False)  # Removing the top spine
    plt.title('Residuals')
    plt.show()  

In [ ]:
homoscedasticity_assumption(pop_model, X_test, y_test)

In [ ]:
regr = DecisionTreeRegressor(max_depth=3, random_state=0)
model = regr.fit(X_train, y_train)

y_predict = regr.predict(X_test)
rms = mean_squared_error(y_test, y_predict, squared=False)

%matplotlib inline
fig = plt.figure(figsize=(18,18))
_ = tree.plot_tree(regr, feature_names=['mrt_counts', 'ps_counts', 'ss_count', 'sm_count', 'com_count'], filled=True)